In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

In [2]:
directory_in = './file/2.url'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：1220



,No,Url
0,1,https://www.dormanproducts.com/p-9421-w37425.a...
1,2,https://www.dormanproducts.com/p-21537-w72258....
2,3,https://www.dormanproducts.com/p-50659-w610165...
3,4,https://www.dormanproducts.com/p-47833-w610150...
4,5,https://www.dormanproducts.com/p-50424-w610166...
...,...,...
1215,1216,https://www.dormanproducts.com/p-6828-w37969.a...
1216,1217,https://www.dormanproducts.com/p-34024-w73620....
1217,1218,https://www.dormanproducts.com/p-15139-m39944....
1218,1219,https://www.dormanproducts.com/p-46918-m630147...


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                     1
 Url    https://www.dormanproducts.com/p-9421-w37425.a...
 Name: 0, dtype: object)

In [5]:
count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_request = crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            type_ = html.xpath('//h1/span[@id="lblPartTypeName"]/text()')[0].strip()
            if type_:
                with open('./part.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        continue

type_

1
2
3
4


'Wheel Cylinder'

In [6]:
dict_src, i, list_href = {}, 0, list(reversed([href.strip() for href in html.xpath('//a[@data-zoom-id="ProductPic"]/@href')]))
for href in list_href:
    if href.startswith('http'):
        dict_src[str(i)] = href
        i += 1

if not list_href:
    list_href = [href.strip() for href in html.xpath('//a[@id="ProductPic"]/@href')]
    if list_href:
        dict_src = {'0': list_href[0]}

dict_src

{'0': 'https://static.dormanproducts.com/images/product/medium/W37425-007.JPG',
 '1': 'https://static.dormanproducts.com/images/product/medium/W37425-006.JPG',
 '2': 'https://static.dormanproducts.com/images/product/medium/W37425-004.JPG'}

In [7]:
part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()

part_number

'W37425'

In [8]:
description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()

description

'Drum Brake Wheel Cylinder'

In [9]:
list_text = [text.strip() for text in html.xpath('//div[@id="divAppSummary"]/p/text()')]
while '' in list_text:
    list_text.remove('')

application_summary = ';'.join(list_text)

application_summary

'BMW 2002 1976-69, BMW 2002ti 1971-69, BMW 2002tii 1974-71'

In [10]:
brand = html.xpath('//span[@id="lblBrandName"]/text()')[0].strip()

brand

'Dorman - First Stop'

In [11]:
value = html.xpath('//input[@id="hidfDetailApp"]/@value')
vehicle_url = f'https://www.dormanproducts.com/{value[0].strip()}' if value else ''

vehicle_url

'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=9421&PartType=Wheel%20Cylinder'

In [12]:
list_tr, dict_count = html.xpath('//section[@id="productSpec"]/div/table/tr'), {}
for tr in list_tr:
    key = tr.xpath('./th/text()')[0].strip().removesuffix(':').strip()
    if key in dict_count:
        dict_count[key] += 1
    else:
        dict_count[key] = 1

dict_specification, dict_no = {}, {}
for i, tr in enumerate(list_tr):
    key = tr.xpath('./th/text()')[0].strip().removesuffix(':').strip()
    if dict_count[key] == 1:
        dict_specification[str(i)] = {key: tr.xpath('./td/text()')[0].strip()}
    else:
        if key in dict_no:
            dict_no[key] += 1
        else:
            dict_no[key] = 1
        dict_specification[str(i)] = {f'{key}-{dict_no[key]}': tr.xpath('./td/text()')[0].strip()}

dict_specification

{'0': {'Bleeder Screw Cap Included': 'No'},
 '1': {'Bleeder Screw Included': 'Yes'},
 '2': {'Brake Component Inlet Thread Size': 'M10 X 1.0'},
 '3': {'Inside Diameter': '0.625 in'},
 '4': {'Material': 'Cast Iron'},
 '5': {'Package Contents': '1 Drum Brake Wheel Cylinder'},
 '6': {'Package Quantity': '1'},
 '7': {'Packaging Type': 'Box'},
 '8': {'Thread Diameter': '10 mm'},
 '9': {'Thread Pitch': '1.0'}}

In [13]:
list_text = [text.strip() for text in html.xpath('//section[@id="productOE"]/div/table/tbody/tr/th/text()')]
oem = ';'.join(sorted(list(set(list_text)), key=list_text.index))

oem

'34 21 1 101 760'

In [14]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Type': type_,
                          'Part Number': part_number,
                          'Brand': brand,
                          'Description': description,
                          'Application Summary': application_summary,
                          'OEM': oem,
                          'Picture': '',
                          'Url': url_request,
                          'Json_Src': json.dumps(dict_src),
                          'Vehicle Url': vehicle_url,
                          'Json_Specification': json.dumps(dict_specification)})

list_dict_correct

[{'No': 1,
  'Type': 'Wheel Cylinder',
  'Part Number': 'W37425',
  'Brand': 'Dorman - First Stop',
  'Description': 'Drum Brake Wheel Cylinder',
  'Application Summary': 'BMW 2002 1976-69, BMW 2002ti 1971-69, BMW 2002tii 1974-71',
  'OEM': '34 21 1 101 760',
  'Picture': '',
  'Url': 'https://www.dormanproducts.com/p-9421-w37425.aspx?origin=keyword',
  'Json_Src': '{"0": "https://static.dormanproducts.com/images/product/medium/W37425-007.JPG", "1": "https://static.dormanproducts.com/images/product/medium/W37425-006.JPG", "2": "https://static.dormanproducts.com/images/product/medium/W37425-004.JPG"}',
  'Vehicle Url': 'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=9421&PartType=Wheel%20Cylinder',
  'Json_Specification': '{"0": {"Bleeder Screw Cap Included": "No"}, "1": {"Bleeder Screw Included": "Yes"}, "2": {"Brake Component Inlet Thread Size": "M10 X 1.0"}, "3": {"Inside Diameter": "0.625 in"}, "4": {"Material": "Cast Iron"}, "5": {"Package Contents": "1 Drum Brake Whee

In [15]:
crawler_status = True

crawler_status

True

In [16]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://www.dormanproducts.com/p-9421-w37425.a...
 Request_Url    https://www.dormanproducts.com/p-9421-w37425.a...
 Name: 0, dtype: object]

In [17]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Type', 'Part Number'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct['No'] = range(1, len(df_correct) + 1)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct

,No,Type,Part Number,Brand,Description,Application Summary,OEM,Picture,Url,Json_Src,Vehicle Url,Json_Specification
0,1,Wheel Cylinder,W37425,Dorman - First Stop,Drum Brake Wheel Cylinder,"BMW 2002 1976-69, BMW 2002ti 1971-69, BMW 2002...",34 21 1 101 760,,https://www.dormanproducts.com/p-9421-w37425.a...,"{""0"": ""https://static.dormanproducts.com/image...",https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": {""Bleeder Screw Cap Included"": ""No""}, ""1..."


In [18]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)

df_error

,No,Url,Request_Url
0,1,https://www.dormanproducts.com/p-9421-w37425.a...,https://www.dormanproducts.com/p-9421-w37425.a...
